In [ ]:
"""
Author: Simon Perrod
Version: 1.0.0
Date: 18/07JULY/2025

Description:
This script analyzes portfolio performance based on historical balance data:
    1. Calculates performance across various timeframes, including:
        - Yesterday (00:00–24:00 UTC)
        - Past N calendar days (00:00–24:00 UTC)
        - Rolling past 24 hours, past X minutes, etc.
    2. Draws equity curves with customized formatting.
"""
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta, timezone
from scipy.interpolate import make_interp_spline
import bisect
import re

pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_rows', 500)

# ========== Load equity data ==========
def load_equity_data_in_range(db_path, days=30, accounts=None):
    now = pd.Timestamp.utcnow().normalize()
    start_time = now - pd.Timedelta(days=days)
    end_time = now

    query_start = int((start_time - pd.Timedelta(minutes=10)).timestamp() * 1000)
    query_end = int((end_time + pd.Timedelta(minutes=10)).timestamp() * 1000)

    with sqlite3.connect(db_path) as conn:
        df = pd.read_sql_query(
            "SELECT * FROM equity_log WHERE timestamp BETWEEN ? AND ?",
            conn,
            params=(query_start, query_end)
        )

    df["timestamp"] = pd.to_datetime(df["timestamp"].astype(float), unit="ms", utc=True)
    df = df.sort_values("timestamp")
    batch_timestamps = sorted(df["timestamp"].unique())

    if accounts:
        cols = ["timestamp"] + accounts
        df = df[cols]
        df = df.melt(id_vars=["timestamp"], var_name="account", value_name="equity")
    else:
        df = df[["timestamp", "total"]]
        df.rename(columns={"total": "equity"}, inplace=True)

    return df, batch_timestamps, start_time, end_time

# ========== Compute equity curve ==========
def find_nearest_timestamp(ts_list, target):
    if not ts_list:
        return None
    index = bisect.bisect_left(ts_list, target)
    if index == 0:
        return ts_list[0]
    if index == len(ts_list):
        return ts_list[-1]
    before = ts_list[index - 1]
    after = ts_list[index]
    return before if abs(before - target) < abs(after - target) else after

def compute_equity_curve(df, batch_timestamps, start_time, end_time, freq="15min", aggregate=True):
    points = pd.date_range(start=start_time, end=end_time, freq=freq)
    grouped = dict(tuple(df.groupby("timestamp")))

    results = []
    for t in points:
        ts = find_nearest_timestamp(batch_timestamps, t)
        if ts is None:
            continue
        snapshot = grouped.get(ts)
        if snapshot is None:
            continue
        equity = snapshot["equity"].sum() if aggregate else snapshot["equity"].iloc[0]
        results.append((t, equity))

    return pd.DataFrame(results, columns=["time", "equity"])

# ========== Analyze equity performance ==========
def analyze_equity_curve(df_curve, initial_capital=0):
    if df_curve.empty:
        print("No data for equity curve analysis.")
        return

    equity_start = df_curve["equity"].iloc[0] + initial_capital
    equity_end = df_curve["equity"].iloc[-1] + initial_capital
    total_return = equity_end - equity_start

    print("--------------------------")
    print(f"Period: {df_curve['time'].iloc[0]} ~ {df_curve['time'].iloc[-1]} UTC")
    print(f"Start Equity: {equity_start:.2f}, End Equity: {equity_end:.2f}, Return: {total_return:.2f}")

# ========== Plotting ==========
def plot_equity_curve(df_curve, title="Equity Curve"):
    if df_curve.empty:
        print("No data to plot.")
        return

    df = df_curve.copy()
    df["time"] = pd.to_datetime(df["time"])
    df = df.sort_values("time")
    df = df.drop_duplicates(subset="time")

    df = df.dropna(subset=["equity"])
    x = mdates.date2num(df["time"].values)
    y = df["equity"].values

    if len(df) >= 4:
        x_new = np.linspace(x.min(), x.max(), 500)
        spline = make_interp_spline(x, y, k=3)
        y_smooth = spline(x_new)
        time_smooth = mdates.num2date(x_new)
    else:
        time_smooth = df["time"]
        y_smooth = y

    # Plot
    plt.figure(figsize=(12, 5), facecolor='white')
    plt.plot(time_smooth, y_smooth, label="Equity", color="steelblue", linewidth=2)
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Equity (USD)")
    plt.tight_layout()
    plt.legend()
    plt.show()

# ========== Run interface ==========
def run_equity_curve_analysis(db_path, days=1, freq="15min", accounts=None, plot=True):
    df, batch_timestamps, start_time, end_time = load_equity_data_in_range(db_path, days=days, accounts=accounts)
    if df.empty or df["equity"].dropna().empty:
        print(f"No valid equity data found for {accounts if accounts else 'total'} in the specified range.")
        return

    df_curve = compute_equity_curve(
        df, batch_timestamps, start_time, end_time,
        freq=freq,
        aggregate=(accounts is None or len(accounts) > 1)
    )

    if df_curve.empty or df_curve["equity"].dropna().empty:
        print(f"Equity curve is empty or invalid for {accounts if accounts else 'total'}")
        return

    analyze_equity_curve(df_curve, initial_capital=0)

    if plot:
        account_label = "All Accounts" if accounts is None else ", ".join(accounts)
        title = f"Total Net Value Curve - {account_label} ({freq} interval)"
        plot_equity_curve(df_curve, title)

    return df_curve

# ========== Yesterday Performance ==========
def get_yesterday_midnight_equity(db_path):
    """
    Fetch the nearest equity snapshot to yesterday midnight, for all 'sixteen***' accounts,
    and insert it into daily_equity_snapshot table.
    """
    now = datetime.now(timezone.utc)
    today_midnight = now.replace(hour=0, minute=0, second=0, microsecond=0)
    target_timestamp_ms = int(today_midnight.timestamp() * 1000)
    date_str = (today_midnight - timedelta(days=1)).strftime("%Y-%m-%d")

    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()

        # Skip if snapshot already exists
        cursor.execute("SELECT 1 FROM daily_equity_snapshot WHERE date = ?", (date_str,))
        if cursor.fetchone():
            print("--------------------------")
            print(f"Snapshot already exists for {date_str}, skipping.")
            return

        # Get all column names from equity_log
        cursor.execute("PRAGMA table_info(equity_log)")
        columns = [row[1] for row in cursor.fetchall()]
        account_cols = [col for col in columns if col.startswith("sixteen")]
        all_cols = ["timestamp"] + account_cols + (["total"] if "total" in columns else [])

        # Find the closest timestamp to midnight
        window_start = target_timestamp_ms - 30 * 60 * 1000
        window_end = target_timestamp_ms + 30 * 60 * 1000
        cursor.execute(
            f"""
            SELECT {', '.join(all_cols)}
            FROM equity_log
            WHERE timestamp BETWEEN ? AND ?
            ORDER BY ABS(timestamp - ?) ASC
            LIMIT 1
            """,
            (window_start, window_end, target_timestamp_ms)
        )
        row = cursor.fetchone()
        if not row:
            print("No equity record near midnight.")
            return

        # Construct insert SQL
        best_ts = row[0]
        insert_cols = ["timestamp", "date"] + account_cols + (["total"] if "total" in columns else [])
        insert_placeholders = ', '.join(['?'] * len(insert_cols))
        insert_sql = f"""
            INSERT OR REPLACE INTO daily_equity_snapshot ({', '.join(insert_cols)})
            VALUES ({insert_placeholders})
        """
        insert_values = [best_ts, date_str] + list(row[1:])
        cursor.execute(insert_sql, insert_values)
        conn.commit()

        print("--------------------------")
        print(f"Midnight Snapshot Inserted for {date_str}")
        print("Accounts:", ", ".join(account_cols))

# ======= Plot Daily Curve =======
def plot_daily_total_equity_curve(db_path, account=None):
    """
    Plot daily equity curve for total or a specific 'sixteen***' account.
    Each account uses a customized start date for display.
    """
    # Define minimum available date per account
    account_start_dates = {
        "sixteen001": "2025-07-16",
        "sixteen002": "2025-07-31",
        "sixteen006": "2025-07-31"
    }

    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("PRAGMA table_info(daily_equity_snapshot)")
        available_cols = [row[1] for row in cursor.fetchall()]

        if account is None and "total" in available_cols:
            df = pd.read_sql_query(
                "SELECT date, total AS value FROM daily_equity_snapshot ORDER BY timestamp ASC", conn
            )
        elif account in available_cols:
            start_date = account_start_dates.get(account, "2025-07-16")
            df = pd.read_sql_query(
                f"""
                SELECT date, {account} AS value
                FROM daily_equity_snapshot
                WHERE date >= ?
                ORDER BY timestamp ASC
                """, conn,
                params=(start_date,)
            )
        else:
            print(f"Unsupported or missing account: {account}")
            return

    df["date"] = pd.to_datetime(df["date"])
    df["value"] = df["value"].astype(float)

    # Require at least 10 days of data to plot
    if account is not None and len(df) < 10:
        print(f"Insufficient data for {account}, only {len(df)} days available. Skipping plot.")
        plot = False
    else:
        plot = True

    navs = df["value"].tolist()
    dates = df["date"].tolist()
    nav_series = pd.Series(navs)

    returns = nav_series.pct_change().dropna()
    annualized_return = (1 + returns.mean()) ** 365 - 1
    annualized_volatility = returns.std() * np.sqrt(365)
    risk_free_rate = 0
    sharpe_ratio = (returns.mean() * 365 - risk_free_rate) / (returns.std() * np.sqrt(365)) if returns.std() > 0 else 0

    downside_returns = returns[returns < 0]
    downside_std = downside_returns.std()
    sortino_ratio = (returns.mean() * 365 - risk_free_rate) / (downside_std * np.sqrt(365)) if downside_std > 0 else 0

    yesterday_return = navs[-1] - navs[-2] if len(navs) >= 2 else 0
    yesterday_return_pct = yesterday_return / navs[0] if navs[0] != 0 else 0
    total_return = navs[-1] - navs[0]
    total_return_pct = total_return / navs[0] if navs[0] != 0 else 0

    peak_series = nav_series.cummax()
    drawdown = (nav_series - peak_series) / peak_series
    min_drawdown = drawdown.min()
    max_nav = nav_series.max()
    start_nav = nav_series[0]
    max_return_abs = max_nav - start_nav
    max_return_pct = max_return_abs / start_nav if start_nav != 0 else 0
    max_dd_abs = (min_drawdown * peak_series).min()

    # Choose display precision based on account
    if account in [None, "sixteen001"]:
        total_str = f"{total_return:.0f}USDT / {total_return_pct:.2%}"
        yesterday_str = f"{yesterday_return:.0f}USDT / {yesterday_return_pct:.2%}"
    else:  # sixteen002
        total_str = f"{total_return:.2f}USDT / {total_return_pct:.4%}"
        yesterday_str = f"{yesterday_return:.2f}USDT / {yesterday_return_pct:.4%}"

    # Print
    print("--------------------------")
    print(f"Account: {account if account else 'Total'}")
    print(f"Period : {df['date'].iloc[0].date()} ~ {df['date'].iloc[-1].date()}")
    print(f"Total  : {total_str}")
    print(f"Yesterday: {yesterday_str}")
    print(f"APY    : {annualized_return * 100:.2f}%")
    print(f"Volatility: {annualized_volatility * 100:.2f}%")
    print(f"Sharpe : {sharpe_ratio:.4f}")
    print(f"Sortino: {sortino_ratio:.4f}")
    print(f"Max Return: {max_return_abs:.0f}USDT / {max_return_pct:.2%}")
    print(f"Max DD    : {max_dd_abs:.0f}USDT / {min_drawdown:.2%}")

    print(f"Total: {total_str}, Yesterday: {yesterday_str}")
    print(f"APY: {annualized_return * 100:.2f}%, Vol: {annualized_volatility * 100:.2f}%, Sharpe: {sharpe_ratio:.4f}, Sortino: {sortino_ratio:.4f}")
    print(f"Max Return: {max_return_abs:.0f}USDT / {max_return_pct:.2%}, Max DD: {max_dd_abs:.0f}USDT / {min_drawdown:.2%}")

    if not plot:
        return

    # Interpolation
    x = np.array(mdates.date2num(dates))
    y = np.array(navs)
    x_new = np.linspace(x.min(), x.max(), 500)
    spl = make_interp_spline(x, y, k=3)
    y_smooth = spl(x_new)
    dates_smooth = mdates.num2date(x_new)

    # Plotting
    fig = plt.figure(figsize=(12, 5), facecolor='#FEF5F0')
    ax = fig.add_subplot(111)
    ax.plot(dates_smooth, y_smooth, color='green', linewidth=2, label='Net Value (USDT)')
    ax.set_facecolor('#FEF5F0')
    ax.set_title(f'Daily Equity Curve - {account if account else "Total"}', fontsize=16)
    ax.set_xlabel('Date', fontsize=13, labelpad=25)
    ax.set_ylabel('Net Value (USDT)', fontsize=13)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray', alpha=0.6)
    ax.set_xticks([])

    y_min, y_max = ax.get_ylim()
    y_offset = (y_max - y_min) * 0.02
    month_locs = [d for d in dates if d.day == 1]
    for loc in month_locs:
        label = loc.strftime('%B')
        ax.text(loc, y_min - y_offset, label,
                fontsize=13, fontweight='bold',
                ha='center', va='top', color='black')
    day_locs = [d for d in dates if d.day in [5, 10, 15, 20, 25] and d not in month_locs]
    for loc in day_locs:
        ax.text(loc, y_min - y_offset, str(loc.day),
                fontsize=10, fontweight='normal',
                ha='center', va='top', color='black')

    ax.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

# Example usage:
if __name__ == "__main__":
    db_path = "equity_monitor.db"

    # Dynamically detect all sixteen*** accounts
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("PRAGMA table_info(equity_log)")
        columns = [row[1] for row in cursor.fetchall()]
        all_accounts = [col for col in columns if col.startswith("sixteen")]

    # Run analysis for each account
    for acct in all_accounts:
        run_equity_curve_analysis(db_path, days=1, freq="15min", accounts=[acct])

    # Total equity
    run_equity_curve_analysis(db_path, days=1, freq="15min")

    # Update yesterday snapshot
    get_yesterday_midnight_equity(db_path)

    # Plot daily curves
    plot_daily_total_equity_curve(db_path)  # total
    for acct in all_accounts:
        plot_daily_total_equity_curve(db_path, account=acct)